In [2]:
import gensim
import pymorphy2

In [14]:
#Now we load 
model = gensim.models.KeyedVectors.load_word2vec_format("G:\\New folder\\ruwikiruscorpora_tokens_elmo_1024_2019\\ruwikiruscorpora_upos_skipgram_300_2_2019\\model.bin", binary=True)
model.init_sims(replace=True)

morph = pymorphy2.MorphAnalyzer()

In [15]:
morph.parse('черт')

[Parse(word='черт', tag=OpencorporaTag('NOUN,inan,femn plur,gent'), normal_form='черта', score=0.588235, methods_stack=((<DictionaryAnalyzer>, 'черт', 55, 8),)),
 Parse(word='чёрт', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='чёрт', score=0.411764, methods_stack=((<DictionaryAnalyzer>, 'чёрт', 3019, 0),))]

In [18]:
model.most_similar(['холодный_ADJ'], topn=20)

[('теплый_ADJ', 0.7089695930480957),
 ('прохладный_ADJ', 0.6719748377799988),
 ('жаркий_ADJ', 0.6454541087150574),
 ('холодная_ADJ', 0.6336202621459961),
 ('ледяной_ADJ', 0.6221423745155334),
 ('влажный_ADJ', 0.6042449474334717),
 ('холод_NOUN', 0.6039727330207825),
 ('горячий_ADJ', 0.5988045334815979),
 ('пронизывающий_ADJ', 0.5900486707687378),
 ('сырой_ADJ', 0.5874677896499634),
 ('зябко_ADJ', 0.5839831829071045),
 ('морозный_ADJ', 0.5822979211807251),
 ('студеный_ADJ', 0.5759075880050659),
 ('горячий_NOUN', 0.5737862586975098),
 ('холодноватый_ADJ', 0.56715989112854),
 ('холодновато_ADJ', 0.5660685300827026),
 ('мокрый_ADJ', 0.5657874345779419),
 ('сухой_ADJ', 0.5657292008399963),
 ('холодно_ADJ', 0.5602232813835144),
 ('ветреный_ADJ', 0.5539122819900513)]

In [19]:
def search_neighbour(word, pos, gend='masc'):
    word = word.replace('ё', 'е')
    lex = word + '_' + cotags[pos]
    if lex in model:
        neighbs = model.most_similar([lex], topn=20)
        for nei in neighbs:
            lex_n, ps_n = nei[0].split('_')
            if '::' in lex_n:
                continue
            if cotags[pos] == ps_n:
                if pos == 'NOUN':
                    parse_result = morph.parse(lex_n)
                    for ana in parse_result:
                        if ana.normal_form == lex_n:
                            if ana.tag.gender == gend:
                                return lex_n
                elif cotags[pos] == 'VERB' and word[-2:] == 'ся':
                    if lex_n[-2:] == 'ся':
                        return lex_n
                elif cotags[pos] == 'VERB' and word[-2:] != 'ся':
                    if lex_n[-2:] != 'ся':
                        return lex_n
                else:
                    return lex_n
    return None

In [20]:
def flection(lex_neighb, tags):
    tags = str(tags)
    tags = re.sub(',[AGQSPMa-z-]+? ', ',', tags)
    tags = tags.replace("impf,", "")
    tags = re.sub('([A-Z]) (plur|masc|femn|neut|inan)', '\\1,\\2', tags)
    tags = tags.replace("Impe neut", "")
    tags = tags.split(',')
    tags_clean = []
    for t in tags:
        if t:
            if ' ' in t:
                t1, t2 = t.split(' ')
                t = t2
            tags_clean.append(t)
    tags = frozenset(tags_clean)
    prep_for_gen = morph.parse(lex_neighb)
    ana_array = []
    for ana in prep_for_gen:
        if ana.normal_form == lex_neighb:
            ana_array.append(ana)
    for ana in ana_array:
        try:
            flect = ana.inflect(tags)
        except:
            print(tags)
            return None
        if flect:
            word_to_replace = flect.word
            return word_to_replace
    return None

In [21]:
cotags = {'ADJF':'ADJ', # pymorphy2: word2vec 
'ADJS' : 'ADJ', 
'ADVB' : 'ADV', 
'COMP' : 'ADV', 
'GRND' : 'VERB', 
'INFN' : 'VERB', 
'NOUN' : 'NOUN', 
'PRED' : 'ADV', 
'PRTF' : 'ADJ', 
'PRTS' : 'VERB', 
'VERB' : 'VERB'}